In [ ]:
!wget https://www.dropbox.com/s/9ikq5yptzqigeso/stacksampleProject1.zip?dl=0

--2023-09-18 12:45:42--  https://www.dropbox.com/s/9ikq5yptzqigeso/stacksampleProject1.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/9ikq5yptzqigeso/stacksampleProject1.zip [following]
--2023-09-18 12:45:43--  https://www.dropbox.com/s/raw/9ikq5yptzqigeso/stacksampleProject1.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce933a5c122fe2b545fc0238b0c.dl.dropboxusercontent.com/cd/0/inline/CD8PuDfjYghrPsYXOj5bKRHD81_6nZVqBLQTSOjGA-5tmnP1AOaM6XSUADgtCwhoGzG5u7taujwA7G6GtyPXG_MTmrM9l0_TKZN-h-33H0kMlzaQ3tzThBg6v504Fs5uOJ3PbP7TchmI2kyFIcFDynMn/file# [following]
--2023-09-18 12:45:43--  https://uce933a5c122fe2b545fc0238b0c.dl.dropboxusercontent.com/cd/0/inline/CD8PuDfjYghrPsYXOj5bKRHD81_6nZVqBLQTSOjGA-5tmnP1AOaM6XSUADgtCwho

In [ ]:
!unzip stacksampleProject1.zip\?dl\=0

Archive:  stacksampleProject1.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
file=r'/content/Questions.csv'
question_data=pd.read_csv(file,encoding='latin-1')

tag_data=pd.read_csv(r'/content/Tags.csv')

In [ ]:
question_data=question_data.sample(10000)

In [ ]:
question_data.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
123915,5025200,76486.0,2011-02-17T04:57:40Z,NaN,2,Invalidate browser cache of back button (bfcac...,<p>I have a form which submits to an iFrame (t...
41580,2040130,202335.0,2010-01-11T05:45:46Z,NaN,1,How to align the top lines of two DIVs?,<p>I want the top lines of two DIVs (<code>&lt...
1240089,39566980,6838564.0,2016-09-19T06:51:00Z,NaN,-1,Starting a WPF application in Hidden mode,<p>I am trying to launch a WPF application fro...
645728,22160100,190623.0,2014-03-03T23:54:31Z,NaN,1,paginating over big number of records,<p>I've got a big number of records and I disp...
555125,19214780,1321251.0,2013-10-06T22:34:54Z,NaN,2,Devise Invitable send different emails based o...,<p>I was using devise_invitable in previous ve...


In [ ]:
tag_id_list=question_data['Id'].to_list()

In [ ]:
top_10_tags=tag_data['Tag'].value_counts().sort_values(ascending=False).index[0:10].tolist()

In [ ]:
top_10_tags

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [ ]:
tag_data=tag_data[tag_data['Tag'].isin(top_10_tags)]

In [ ]:
x=pd.crosstab(index=tag_data['Id'], columns=tag_data['Tag']).reset_index()

In [ ]:
x.tail()

Tag,Id,android,c#,c++,html,ios,java,javascript,jquery,php,python
706331,40143150,0,0,0,0,0,0,1,0,0,0
706332,40143190,0,0,0,0,0,0,0,0,0,1
706333,40143210,0,0,0,0,0,0,0,0,1,0
706334,40143340,1,0,0,0,0,0,0,0,0,0
706335,40143360,0,0,0,0,0,0,1,0,0,0


In [ ]:
data=question_data.merge(x,on='Id',how='inner')

In [ ]:
def process_data(txt):

  # remove code snippets, HTML TAGS, Links
  clean_text = re.sub(r"<pre><code>.*?</code></pre>",r"CODE_SNIPPET",str(txt))
  clean_text = re.sub(r"<a[^>]*>(.*?)</a>", r"\1", clean_text)
  clean_text = re.sub(r"https?://\S+", "", clean_text)
  clean_text = re.sub(r"<.*?>", " ", clean_text)
  clean_text = " ".join(clean_text.split())

  # special character and punctuation
  clean_text = re.sub(r"[^A-Za-z0-9\s]+", "", clean_text)

  # reomoving numbers
  clean_text = re.sub(r"\b[0-9]+\b\s*", "", clean_text)

  # removing stop-words
  stopwords_ = set(stopwords.words("english"))
  tokens = clean_text.split()
  clean_tokens = [t for t in tokens if t.lower()=='c' or not t in stopwords_]
  clean_text = " ".join(clean_tokens)

  # SnowBall Stemming
  snowball = SnowballStemmer(language='english')
  text=''
  for word in clean_text.split():
    text=text+snowball.stem(word)+' '

  return text

In [ ]:
data['Body']=data['Body'].apply(process_data)
data['Title']=data['Title'].apply(process_data)

In [ ]:
data

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,android,c#,c++,html,ios,java,javascript,jquery,php,python
0,5025200,76486.0,2011-02-17T04:57:40Z,NaN,2,invalid browser cach back button bfcach post i...,i form submit ifram common approach ajax imag ...,0,0,0,0,0,0,0,1,0,0
1,2040130,202335.0,2010-01-11T05:45:46Z,NaN,1,how align top line two div,i want top line two div ltdivgtltdivgt align h...,0,0,0,1,0,0,0,0,0,0
2,39566980,6838564.0,2016-09-19T06:51:00Z,NaN,-1,start wpf applic hidden mode,i tri launch wpf applic anoth wpf applic i wan...,0,1,0,0,0,0,0,0,0,0
3,7466730,948644.0,2011-09-19T05:19:02Z,NaN,1,automat updat data tabl,in window applic i feel data tabl time page lo...,0,1,0,0,0,0,0,0,0,0
4,9550220,1234275.0,2012-03-03T21:54:48Z,NaN,0,fade doesnt work javascript,it simpl task somewher mistak i cant find func...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5566,37991130,6389176.0,2016-06-23T12:04:30Z,NaN,0,handl facebook loginmanag logout,im tri logout get user detail im tri achiev ki...,1,0,0,0,0,0,0,0,0,0
5567,36472330,6061978.0,2016-04-07T09:41:29Z,NaN,0,what purpos restrict access jre librari class ...,ive recent tri use class appengin restrict spe...,0,0,0,0,0,1,0,0,0,0
5568,32342840,1024941.0,2015-09-02T00:21:48Z,2015-09-02T13:15:20Z,-2,how i select nest element insid first child el...,i follow html ltdiv classcontainergt ltdivgt l...,0,0,0,1,0,0,0,0,0,0
5569,9988810,861679.0,2012-04-03T07:22:21Z,NaN,0,android setvis issu android,just straight forward problem i linearlayout a...,1,0,0,0,0,0,0,0,0,0


In [ ]:
input_data=data[['Title','Body']]
output_data=data[top_10_tags]
X_train, X_test, y_train, y_test = train_test_split(input_data,output_data,train_size=0.8)
X_train["Title_Body"] = X_train["Title"].str.cat(X_train["Body"], sep = " ")

In [ ]:
tfidf=TfidfVectorizer(analyzer='word',stop_words='english',min_df=0.01,max_df=0.8)
tfidf.fit(X_train['Title_Body'])
X_train=tfidf.transform(X_train['Title_Body'])

In [ ]:
X_test["Title_Body"] = X_test["Title"].str.cat(X_test["Body"], sep = " ")
X_test=tfidf.transform(X_test['Title_Body'])

In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import hamming_loss,accuracy_score,f1_score

In [ ]:
print("accuracy :",accuracy_score(y_test,predictions))
print("macro f1 score :",f1_score(y_test, predictions, average = 'macro'))
print("micro f1 scoore :",f1_score(y_test, predictions, average = 'micro'))
print("hamming loss :",hamming_loss(y_test,predictions))

accuracy : 0.5067264573991032
macro f1 score : 0.594841737581923
micro f1 scoore : 0.568515497553018
hamming loss : 0.09488789237668162


In [ ]:
predictions

<1115x10 sparse matrix of type '<class 'numpy.float64'>'
	with 1174 stored elements in Compressed Sparse Column format>

In [ ]:
predictions=pd.DataFrame.sparse.from_spmatrix(predictions,columns=y_test.columns)

In [ ]:
y_test=y_test.reset_index(drop=True)

In [ ]:
predictions.compare(y_test,keep_equal=True)

javascript       java         c#        php       android       jquery  \
           self other self other self other self other    self other   self   
0           0.0     0  0.0     0  0.0     1  0.0     0     0.0     0    0.0   
2           0.0     1  0.0     0  0.0     0  0.0     0     0.0     0    0.0   
5           0.0     0  0.0     0  0.0     0  0.0     0     0.0     0    0.0   
6           0.0     0  0.0     1  1.0     0  0.0     0     0.0     0    0.0   
7           0.0     0  0.0     0  0.0     1  0.0     0     0.0     0    0.0   
...         ...   ...  ...   ...  ...   ...  ...   ...     ...   ...    ...   
1102        0.0     0  0.0     0  0.0     0  0.0     0     0.0     0    0.0   
1105        0.0     0  0.0     0  0.0     1  0.0     0     0.0     0    0.0   
1109        0.0     0  0.0     0  0.0     0  0.0     1     0.0     0    0.0   
1113        0.0     1  0.0     0  0.0     0  0.0     0     0.0     0    0.0   
1114        0.0     1  0.0     0  0.0     0  0.0     0     0.0     0    0.0   

           python       html        c++        ios        
     other   self other self other self other self other  
0        0    0.0     0  0.0     0  1.0     0  0.0     0  
2        0    0.0     0  0.0     0  0.0     0  1.0     0  
5        0    0.0     0  0.0     0  0.0     1  1.0     0  
6        0    0.0     0  0.0     0  0.0     0  0.0     0  
7        0    0.0     0  0.0     0  0.0     0  1.0     0  
...    ...    ...   ...  ...   ...  ...   ...  ...   ...  
1102     0    1.0     0  0.0     0  0.0     1  0.0     0  
1105     0    0.0     0  0.0     0  0.0     0  1.0     0  
1109     0    0.0     0  0.0     0  0.0     0  1.0     0  
1113     0    0.0     0  0.0     0  0.0     0  1.0     0  
1114     0    0.0     0  0.0     0  0.0     0  1.0     0  

[550 rows x 20 columns]

In [ ]:
cols=y_test.columns.to_list()

In [ ]:
def print_tags(text1,text2):
  ext_test=[{"Title":text1,"Body":text2}]
  ext_test=pd.DataFrame.from_dict(ext_test)
  ext_test["Title_Body"] = ext_test["Title"].str.cat(ext_test["Body"], sep = " ")
  ext_test['Title_Body']=ext_test['Title_Body'].apply(process_data)
  ext_test=tfidf.transform(ext_test['Title_Body'])
  pred=classifier.predict(ext_test).toarray()
  return pred

  # a=[cols[i] for i in range(10) if int(pred[0][i])==1]

  # for i in a:
  #   print(i)

In [ ]:
print(print_tags("a","My dad called today said people going website getting viruses trying download computers He isnt technical built whole thing WYSIWYG editor I popped site open viewed source line Javascript includes bottom source right closing HTML tag They included file among many others lt TURN OFF JAVASCRIPT BEFORE YOU GO TO THAT URL So I commented It turns ftp password plain dictionary word six letters long think thats got hacked Weve changed password digit nonword string wouldnt go passphrase since huntnpeck typer I whois 98hsru found hosted server Chile There actually email address associated I seriously doubt person culprit Probably site got hacked I idea point though Ive never dealt sort thing Anyone suggestions He using plain jane unsecured ftp webhost4lifecom I dont even see way sftp site Im thinking username password got intercepted So make relevant community steps takebest practices follow protect website getting hacked For record line code magically got added file isnt file computer Ive left commented make absolute sure wont anything page although Im sure Jeff would guard CODESNIPPET"))

ios
None


In [ ]:
import pickle

In [ ]:
pickle.dump(tfidf,open('tfidf.pkl','wb'))

In [ ]:
pickle.dump(classifier,open('classifier.pkl','wb'))